## 타이타닉 데이터 전처리 

In [135]:
#필요한 패키지들 가져오기 
import numpy as np #수치연산, 선형대수, ndarray라는 자료 구조를 가진 패키지
import pandas as pd #Series, DF 자료구조를 가 패키지
import seaborn as sns #샘플데이터와 시각화

#1. 타이타닉 데이터 seaborn 패키지 명령어로 가져오기
titanic = sns.load_dataset('titanic')
titanic.info()
print("")

#2. NaN을 소유하고 있는지 확인하기
#2-1. 앞의 10개의 데이터가 NaN을 포함하는지 확인
print(titanic.head(10).isnull())
print("")

#2-2 NaN을 포함한 행의 개수를 파악하기
print(titanic.isnull().sum()) #열로 파악하고 싶다면 sum(axis=1)
print("")

#NaN가 25% 이상인 열은 제거하겠다.
titanic.dropna(thresh=(len(titanic)/4), axis=1, inplace=True)
print(titanic.info())
print("")

#2-3. age열의 값이 NaN인 행을 제거
titanic.dropna(subset=['age'], how='any', axis=0, inplace=True)
print(titanic.info())
print("")

#3. 삭제 대신 치환으로 작업해보기 
titanic = sns.load_dataset('titanic')
print(titanic['embarked'][820:830])
print("")

#3-1. 앞의 값으로 NaN값을 채워보기
titanic['embarked'].fillna(method='ffill', inplace=True)
print(titanic['embarked'][820:830])
# 셀 병합이 된 경우에 사용 할 수 있음

#3-2. 사이킷런으로 결측 값 채워보기
features = np.array([[100],[200],[300],[500],[np.NaN]])

#3-3. 평균으로 채워주는 Imputer를 만들어보자
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean') #mean대신 mode나 median도 사용 가능

#3-4. 3-2에서 만든 features에 imputer를 넣어보자
features_imputed = imputer.fit_transform(features)
print(features_imputed) #상단에서 NaN으로 만든 값이 평균 값으로 치환된 걸 확인 가능

#4. KNN(군집) 알고리즘을 이용한 결측치 채우기
from fancyimpute import KNN
# fancyimpute 없으면 설치하고 오기
features2 = np.array([200,300],[100,200],[300,400],[400,500],[300,200],[105,np.NaN])
features2_imputed = KNN(k=5, verbose=0).fit_transform(featrues2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     Fa

ModuleNotFoundError: No module named 'fancyimpute'

### 중복데이터 처리

In [44]:
# 중복 데이터 처리 하기
df = pd.DataFrame([['두산','SK','두산','롯데','삼성','한화','NC','기아'],['서울','인천','서울','부산','대구','대전','창원','광주']])
df=df.T #행과 열을 뒤바꾸기 (행단위로 비교하기 때문)

#1. 중복 데이터를 먼저 확인한다
df.duplicated()

#2. 중복 데이터 제거해버리기
df.drop_duplicates(inplace=True)
df

,0,1
0,두산,서울
1,SK,인천
3,롯데,부산
4,삼성,대구
5,한화,대전
6,NC,창원
7,기아,광주


### 자료형 변환

In [109]:
#1. auto-mpg_1.csv 파일 불러오기
df = pd.read_csv('./data/auto-mpg_1.csv', header=None) #기존 자료에 헤더가 없거나 바꾸고자 하는 경우 사용 가능한 옵션
df.columns = ['mpg', 'cylinder', 'displacement','horsepower','weight','acceleration','year','origin','name']
print(df.info())
print(df.head())
print("")

#2. origin의 값 바꿔보기
# 1 => 미국, 2 => 유럽, 3 => 일본 으로 치환하고 자료타입을 int64 > object로 바꾸어보자

df['origin'].replace({1:'USA', 2:'EU', 3:'JPN'}, inplace=True)
df['origin'] = df['origin'].astype('category')
#df['origin'] = df['origin'].astype('str')  #다시 문자열로 바꾸고 싶다면 str으로. 
print(df.info())
print("")

#3. Continuous to Discrete
# >> displacement를 대형, 중형, 소형으로 바꾸어보자!

#3-1. 3등분 할 숫자 배열을 만들어 준다.
count, bin_dividers = np.histogram(df['displacement'],bins=4) 
print(bin_dividers)
#bins가 3이면 최소값 ~ 최대값 까지 interval이 3개다. 즉 점은 4개가 필요하다(names와 개수를 맞춘다 생각하자).

#3-2 치환할 데이터를 만들어준다
bin_names = ['소형','중형','대형','초대형']

#3-3 치환 (구간분할 하기)
df['displacement'] = pd.cut(x = df['displacement'], bins=bin_dividers, labels=bin_names, include_lowest=True)
df

dummy=pd.get_dummies(df['displacement']) #get_dumiies()라는 함수를 이용해 원핫 인코딩을 하면 컬럼에 나올 수 있는 모든 값을 조사해서 가능하면 1. 없으면 0.
print(dummy)


#4. Numpy의 digitize를 이용한 변환
age = np.array([[30],[40],[29],[50],[29],[31]])
print(np.digitize(age,bins=[30, 40])) #0~30, 30~40, 40~ 으로 구간을 나누어 줌

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
mpg             398 non-null float64
cylinder        398 non-null int64
displacement    398 non-null float64
horsepower      398 non-null object
weight          398 non-null float64
acceleration    398 non-null float64
year            398 non-null int64
origin          398 non-null int64
name            398 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB
None
    mpg  cylinder  displacement horsepower  weight  acceleration  year  \
0  18.0         8         307.0      130.0  3504.0          12.0    70   
1  15.0         8         350.0      165.0  3693.0          11.5    70   
2  18.0         8         318.0      150.0  3436.0          11.0    70   
3  16.0         8         304.0      150.0  3433.0          12.0    70   
4  17.0         8         302.0      140.0  3449.0          10.5    70   

   origin                       name  
0       1  chevrole

### 여러 개의 열로 구성된 데이터의 이산화 - KMeans Clustering

In [134]:
sample = np.array([[20,30],[30,50],[50,70],[40,20],[23,14],[30,42],[30,64],[64,42]])
df = pd.DataFrame(sample)
df

#1. KMeans 군집분석을 위한 라이브러리 불러오기
from sklearn.cluster import KMeans

#1-1. 군집분석 알고리즘이 들어간 객체를 생성
cluster = KMeans(3, random_state=0) 

#1-2. 데이터를 가지고 fit(훈련)을 시킴
cluster.fit(sample)

#1-3. 예측한 군집 결과를 데이터프레임에 그룹 열로 할당
df['group'] = cluster.predict(sample)
df

,0,1,group
0,20,30,0
1,30,50,1
2,50,70,2
3,40,20,0
4,23,14,0
5,30,42,1
6,30,64,1
7,64,42,2


### 사이킷 런을 이용한 원핫인코딩

In [127]:
from sklearn.preprocessing import LabelBinarizer

one_hot = LabelBinarizer()
#print(one_hot.fit_transform(df['displacement'])) 

#데이터를 정렬하기 때문에 순서를 확인
#print(one_hot.classes_)
#print(one_hot.inverse_transform(one_hot.fit_transform(df['displacement'])))

#여러 개의 특성을 원핫 인코딩
from sklearn.preprocessing import MultiLabelBinarizer
features = [('java','c++'),('java','python'),('c#','r'),('python','r')]
one_hot = MultiLabelBinarizer()
print(one_hot.fit_transform(features))
print(one_hot.classes_)


#get_dummies는 하나의 특성을 하나의 컬럼으로 생성
#값의 종류가 15가지이면 15개의 컬럼을 생성
# (solution) 컬럼은 1개만 만들고 0부터 일련번호 형태로 값을 설정
from sklearn.preprocessing import LabelEncoder
one_hot = LabelEncoder()
print(one_hot.fit_transform(df['displacement']))
print(one_hot.classes_)

[[0 1 1 0 0]
 [0 0 1 1 0]
 [1 0 0 0 1]
 [0 0 0 1 1]]
['c#' 'c++' 'java' 'python' 'r']


KeyError: 'displacement'

### 순서가 있는 범주형 데이터 인코딩 - Ordinal Encoder

In [145]:
#sklearn의 인코더들은 문자열을 기준으로 정렬을 한 후 수치를 부여. 원하는 수치값을 부여하는 것은 불가능
#범주형 데이터에 원하는 수치값을 부여해서 인코딩 할 때는 replace 메소드나 OrdinalEncoder를 이용
df = pd.DataFrame({"Score":['A','B','C','D','F']})
print(df)

#A:4.0, #B:3.0 #C:2.0 #D:1.0 #F : 0
mapper = {'A':4.0,'B':3.0,'C':2.0,'D':1.0,'F':0}

df['encoder'] = df['Score'].replace(mapper)
print(df)


#순서가 있는 범주형 인코딩
from sklearn.preprocessing import OrdinalEncoder

features = np.array([['서울',2],['부산',52],['경기',31],['대구',54]])

#1. 객체를 생성하자
encoder = OrdinalEncoder()

#2.객체에다가 데이터를 준다
result = encoder.fit_transform(features)
print(result) #1열은 가나다순 정렬. 2열은 숫자 정렬

  Score
0     A
1     B
2     C
3     D
4     F
  Score  encoder
0     A      4.0
1     B      3.0
2     C      2.0
3     D      1.0
4     F      0.0
[[3. 0.]
 [2. 2.]
 [0. 1.]
 [1. 3.]]


### 범주형 데이터에서 누락값 삭제 - 머신러닝 알고리즘 이용_최근이웃법

In [168]:
#1.K평균분류 패키지 import
from sklearn.neighbors import KNeighborsClassifier

#훈련할 데이터 생성
X = np.array([[0, 2.10, 14.1],[1,1.18,13.33],[0, 1.22,1.27],[1,-0.10,-14.45]])

#NaN을 가진 데이터
X_with_nan = np.array([[np.NaN,0.87,13.31],[np.NaN, -0.67, -0.22]])

#2. 분류기를 생성 
clf = KNeighborsClassifier(3,weights='distance')

#3. 훈련할 데이터 할당 : 1번째 이후 전체 데이터를 가지고 0번째 데이터를 예측
train_model = clf.fit(X[:,1:],X[:,0])

#4. 데이터 예측
imputed_values = train_model.predict(X_with_nan[:,1:]) #X_with_nan에 행 전체에 1열 이후로 값들을 기반으로 예측해라 
imputed_values

array([1., 0.])

### DataFrame결합 : merge()

In [18]:
#필요한 패키지들 가져오기 
import numpy as np #수치연산, 선형대수, ndarray라는 자료 구조를 가진 패키지
import pandas as pd #Series, DF 자료구조를 가진 패키지
import seaborn as sns #샘플데이터와 시각화

#1. 합쳐줄 파일들 불러오기
price = pd.read_excel('./data/stock price.xlsx')
price.info()
valuation = pd.read_excel('./data/stock valuation.xlsx')
valuation.info()

#2. 두 DataFrame을 id 기준으로 결합하기
merge_df = pd.merge(price,valuation) #두 DataFrame에 중복되는 column이 id이므로 매개변수로 두 DF를 때려넣으면 알아서 매핑해서 결합함
merge_df.info()
#둘다 10개씩인데 왜 merge결과는 5개인가?  inner join 개념이기 때문(양쪽에 다 있어야 merger가 됨)

#3 outer join해보기 : 어느 한쪽에만 존재해도 join에 참여
merge_outer = pd.merge(price,valuation, how='outer', on='id')
merge_outer.info() #>> id가 15개로 바뀜

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
id            10 non-null int64
stock_name    10 non-null object
value         10 non-null float64
price         10 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 448.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
id      10 non-null int64
name    10 non-null object
eps     10 non-null float64
bps     10 non-null int64
per     10 non-null float64
pbr     10 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 608.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 9 columns):
id            5 non-null int64
stock_name    5 non-null object
value         5 non-null float64
price         5 non-null int64
name          5 non-null object
eps           5 non-null float64
bps           5 non-null int64
per           5 non-null float64
pbr           

### DataFrame결합 : join

In [24]:
# join을 쓰는 경우, 파일을 불러오는 단계에서 미리 index를 설정해버린다.
price = pd.read_excel('./data/stock price.xlsx', index_col='id')
valuation = pd.read_excel('./data/stock valuation.xlsx', index_col='id')
price.info()
valuation.info()

stock_join = price.join(valuation)
stock_join # price를 기준으로 valuation에도 있으면 join. 즉, valuation row는 안들어 갈 수도 있음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 128940 to 204210
Data columns (total 3 columns):
stock_name    10 non-null object
value         10 non-null float64
price         10 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 320.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 130960 to 207940
Data columns (total 5 columns):
name    10 non-null object
eps     10 non-null float64
bps     10 non-null int64
per     10 non-null float64
pbr     10 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 480.0+ bytes


,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068.0,15.695091,1.829178
138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN
139480,이마트,239230.833333,254500,이마트,18268.166667,295780.0,13.931338,0.860437
142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090.0,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684.0,25.185866,2.470259
192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN
199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN


### DataFrame 결합 : concat(), append(), combine_first()

In [38]:
#concat? 열 또는 행 방향으로 DataFrame을 합쳐주는 함수
df1 = pd.DataFrame({'a':['a0','a1','a2']}, index=[1,2,3])
df2 = pd.DataFrame({'a':['a2','a3','a4'], 'b':['b2','b3','b4']}, index=[2,3,4])

print(pd.concat([df1,df2],axis=0)) #열이 같다면 행으로 합쳐! 
print(pd.concat([df1,df2],axis=1)) #행이 같다면 열로 합쳐!
print(pd.concat([df1,df2],axis=1, join='inner')) #행이 같다면 열로 합쳐! 근데 한쪽에만 있는것은 다 빼

print(df1.append(df2)) #열이 같다면 행으로 무조건 합치기 
print(df1.combine_first(df2)) #인덱스를 기준으로 합치는데 호출하는 쪽 데이터를 우선적용함

    a    b
1  a0  NaN
2  a1  NaN
3  a2  NaN
2  a2   b2
3  a3   b3
4  a4   b4
     a    a    b
1   a0  NaN  NaN
2   a1   a2   b2
3   a2   a3   b3
4  NaN   a4   b4
    a   a   b
2  a1  a2  b2
3  a2  a3  b3
    a    b
1  a0  NaN
2  a1  NaN
3  a2  NaN
2  a2   b2
3  a3   b3
4  a4   b4
    a    b
1  a0  NaN
2  a1   b2
3  a2   b3
4  a4   b4


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
